In [ ]:
#1
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import cv2

# Initialize YOLO model
model = YOLO("best.pt")


# Set up your video source
cap = cv2.VideoCapture(0)
classes_to_count = [0, 2,18] 
# Initialize Object Counter for your line
line_pedestrian_1 =  [(100, 100), (600, 100), (600, 360), (100, 360)]  # Define as per your need
counter = object_counter.ObjectCounter()
counter.set_args(view_img=True, reg_pts=line_pedestrian_1, classes_names=model.names)

while cap.isOpened():
    ret, im0 = cap.read()
    if not ret:
        break

    # Track objects
    results = model.track(im0,persist=True, show=False, classes=classes_to_count)

    # Start counting based on tracks and your line
    im0 = counter.start_counting(im0, results)

    # Display
    cv2.imshow("Count", im0)
    if cv2.waitKey(1) == ord('q'):  # press q to quit
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
#1
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import cv2

# Initialize YOLO model
model = YOLO("best.pt")


# Set up your video source
cap = cv2.VideoCapture(0)
classes_to_count = [0, 2,18] 
# Initialize Object Counter for your line
line_pedestrian_1 =  [(100, 100), (600, 100), (600, 360), (100, 360)]  # Define as per your need
counter = object_counter.ObjectCounter()

#counter.set_args(view_img=True, reg_pts=line_pedestrian_1, classes_names=model.names)
unique_ids = set()
while cap.isOpened():
    ret, im0 = cap.read()
    if not ret:
        break

    # Track objects
    results = model.track(im0,persist=True, show=False, classes=classes_to_count)

    
    unique_ids.update([track.id for track in results.boxes])
    
    
    # Start counting based on tracks and your line
    im0 = counter.start_counting(im0, results)
    
    # Display
    cv2.imshow("Count", im0)
    if cv2.waitKey(1) == ord('q'):  # press q to quit
        break

cap.release()
cv2.destroyAllWindows()
people_count = len(unique_ids)
print(f"Total unique people counted: {people_count}")

In [ ]:
from ultralytics import YOLO
from ultralytics.solutions import object_counter
import cv2

# Load your model
model = YOLO("best.pt")
cap = cv2.VideoCapture("videoplayback.mp4")
assert cap.isOpened(), "Error reading video file"

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
counter = object_counter.ObjectCounter()
reg_pts = [(0, 0), (frame_width, 0), (frame_width, frame_height), (0, frame_height)]
counter.set_args(view_img=True, classes_names=model.names, draw_tracks=False, reg_pts=reg_pts)
classes_to_count = [0, 2, 18]
class_counts = {0: 0, 16: 0, 15: 0}

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        break  # End of video
    tracks = model.track(im0, persist=True, show=False)
    
    # Update counts
    for track in tracks:
        cls_id = track.cls  # This is how you get class ID
        if cls_id in class_counts:
            class_counts[cls_id] += 1  # Increment count

# After the loop, print your counts
print(f"Person count: {class_counts[0]}, Cat count: {class_counts[15]}, Dog count: {class_counts[16]}")

In [ ]:
from ultralytics import YOLO
import cv2

# Initialize your model
model = YOLO("best.pt")
cap = cv2.VideoCapture(0)
# Assuming cap is your cv2.VideoCapture object for your video stream
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break 
    
    # Perform object tracking
    results = model.track(frame, persist=True)

    if isinstance(results, list):
        print("Object tracking might not be enabled. Check Ultralytics documentation for requirements.")
        continue  # Skip to next iteration if results is a list

    for track in results.tracks:  # Access tracks if available
        class_id = track.class_id
        class_name = model.names[class_id]
        print(class_name)
    
    if cv2.waitKey(1) == ord('q'):  # Press 'q' to quit the video frame
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import json
from ultralytics import YOLO
from collections import defaultdict
obj_name={}
def test_yolo(image_path):
    model = YOLO("best.pt")
    results = model(image_path)
    return results


if __name__ == "__main__":
    results = test_yolo('/content/ddrive/MyDrive/Project/WhatsApp Image 2024-02-29 at 6.38)
    count = 0
    results_array = json.loads(results[0].tojson())



    object_counts = defaultdict(int)

    # Count occurrences of each object name
    for result in results_array:
        if result:
            object_counts[result['name']] += 1


print(object_counts.items())

In [1]:
import json
import cv2
from ultralytics import YOLO
from collections import defaultdict

x1 = 100
y1 = 50
x2 = 500
y2 = 400
model = YOLO("best.pt")  # Load your YOLO model

cap = cv2.VideoCapture(0)  # Open the webcam
a=0
while True:
    ret, frame = cap.read()  # Capture a frame
    roi = frame[y1:y2, x1:x2]
    results = model(roi)  # Run object detection on the frame
    results_array = json.loads(results[0].tojson())
    #print(results_array)

    object_counts = defaultdict(int)

    for result in results_array:
        if result:
            object_counts[result['name']] += 1
    

    # Display object counts on the frame
    for obj_name, count in object_counts.items():
        cv2.putText(frame, f"{obj_name}: {count}", (10, 30 + 30 * len(object_counts)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imshow("Object Counting", frame)  # Display the frame with counts
    a=1
    if object_counts:
        i = object_counts
        cou = list(i.values())[0]
        item_name=list(i.keys())
        count=i.values()
        print(cou)
        print(item_name)
        if cou<3 and item_name[0]=='Pepsi Can':
            break
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


#model = YOLO("best.pt")
p=0
item_names=model.names
for i in item_name:
    for class_id, class_name in item_names.items():
        if i==class_name:
            p=class_id
            print(class_name)
            print(p)

def working(p):
    print(p)
    if p==26:
        print("hee")
        with open('value.txt', 'w') as file:
            # Write new content to the file
            file.write("200")
working(p)



0: 576x640 (no detections), 314.1ms
Speed: 12.4ms preprocess, 314.1ms inference, 1489.9ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 255.3ms
Speed: 0.0ms preprocess, 255.3ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 269.7ms
Speed: 1.6ms preprocess, 269.7ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 268.2ms
Speed: 0.0ms preprocess, 268.2ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 274.7ms
Speed: 0.0ms preprocess, 274.7ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 268.7ms
Speed: 4.2ms preprocess, 268.7ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 269.5ms
Speed: 0.0ms preprocess, 269.5ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 267.3ms
Speed: 0.0ms p


0: 576x640 (no detections), 274.4ms
Speed: 1.9ms preprocess, 274.4ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 276.4ms
Speed: 3.9ms preprocess, 276.4ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 1 Tropicana Smooth-Juice, 291.9ms
Speed: 4.4ms preprocess, 291.9ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 640)
1
['Tropicana Smooth-Juice']

0: 576x640 (no detections), 288.6ms
Speed: 2.9ms preprocess, 288.6ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 290.6ms
Speed: 3.2ms preprocess, 290.6ms inference, 1.4ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 301.3ms
Speed: 5.2ms preprocess, 301.3ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 272.6ms
Speed: 0.0ms preprocess, 272.6ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no d

Speed: 3.2ms preprocess, 268.1ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 264.4ms
Speed: 0.0ms preprocess, 264.4ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 288.2ms
Speed: 2.1ms preprocess, 288.2ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 280.9ms
Speed: 2.7ms preprocess, 280.9ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 267.5ms
Speed: 0.0ms preprocess, 267.5ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 277.8ms
Speed: 3.0ms preprocess, 277.8ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 263.0ms
Speed: 0.0ms preprocess, 263.0ms inference, 0.0ms postprocess per image at shape (1, 3, 576, 640)

0: 576x640 (no detections), 282.8ms
Speed: 4.6ms preprocess, 282.8ms inference, 0.0ms postp

In [2]:
i=object_counts
cou = list(i.values())[0]
print(cou)
item_name=list(i.keys())
count=i.values()
print(item_name)

IndexError: list index out of range

In [3]:
print(item_name)

['Tropicana Smooth-Juice']


In [4]:
model = YOLO("best.pt")

In [5]:
with open('value.txt', 'w') as file:
    # Write new content to the file
    file.write("20")